In [ ]:
import os
import json
import time
from pathlib import Path
import google.generativeai as genai
from google.colab import userdata
from tqdm.auto import tqdm


GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY_TAVE")
genai.configure(api_key=GOOGLE_API_KEY)

MODEL_NAME = "models/gemini-2.5-flash"

BASE_PATH = "/content/drive/MyDrive/TAVE 16th 심화플젝/data/split_scenes/clips_2_faceLabeled2"
OUTPUT_PATH = "/content/drive/MyDrive/TAVE 16th 심화플젝/data/video_captions_final"
os.makedirs(OUTPUT_PATH, exist_ok=True)

START_SCENE = 12
END_SCENE = None
SLEEP_BETWEEN = 2    # 요청 간 텀(초)


# 유틸
def extract_scene_number(video_path: str) -> str:
    filename = os.path.basename(video_path)
    if "Scene-" in filename:
        return filename.split("Scene-")[-1].split("_")[0]
    return "unknown"

def extract_scene_int(video_path: str):
    s = extract_scene_number(video_path)
    try:
        return int(s)
    except:
        return None

def sort_key_by_scene(path: Path):
    n = extract_scene_int(str(path))
    if n is None:
        return (1, path.name)
    return (0, n)

def parse_caption_to_json(caption_text: str):
    txt = caption_text.strip()
    if "```json" in txt:
        txt = txt.split("```json", 1)[1].split("```", 1)[0].strip()
    elif "```" in txt:
        txt = txt.split("```", 1)[1].split("```", 1)[0].strip()
    return json.loads(txt)


# 캡션 생성
def generate_video_caption(video_path: str, scene_number: str) -> str:
    video_file = None
    try:
        model = genai.GenerativeModel(MODEL_NAME)

        video_file = genai.upload_file(path=video_path)

        while video_file.state.name == "PROCESSING":
            time.sleep(5)
            video_file = genai.get_file(video_file.name)

        if video_file.state.name == "FAILED":
            raise RuntimeError("비디오 처리 실패")

        prompt = f"""
이 영상은 영화 장면입니다.
장면 검색(Scene Retrieval)에 활용할 수 있도록
**시각 + 음성 + 내러티브를 모두 포함한 상세 캡션**을 작성하세요.

[규칙]
1) 바운딩 박스에 '배역명(배우명)'이 적혀있으면 그대로 사용
2) 이름이 "unknown" 또는 읽기 어려운 경우 → 이름을 언급하지 말고 '한 남성', '한 여성', '한 사람' 등으로 표현
3) 바운딩박스, 숫자(정확도), 색상, 좌표 등 라벨 표시 방식에 대한 언급 금지.
4) 상상하지 말고 관찰된 것만 기술
5) 인물이 하는 행동, 장면 분위기, 현재 장소 중심으로 보이는 사실만 객관적으로 묘사

[출력 JSON 형식]
{{
  "scene_number": "{scene_number}",
  "characters": [],
  "location": "",
  "actions": [],
  "dialogue": "",
  "visual_details": "",
  "audio_details": "",
  "mood": "",
  "narrative": "",
  "keywords": [],
  "detailed_caption": ""
}}

한국어로 작성하세요.
"""
        response = model.generate_content([video_file, prompt])
        return response.text

    except Exception as e:
        return f"ERROR: {str(e)}"

    finally:
        if video_file is not None:
            try:
                genai.delete_file(video_file.name)
            except:
                pass

# 메인 파이프라인
def process_videos_range():
    all_videos = sorted(Path(BASE_PATH).glob("*.mp4"), key=sort_key_by_scene)
    # scene_number 숫자 있는 것만 범위 필터
    target = []
    for p in all_videos:
        n = extract_scene_int(str(p))
        if n is None:
            continue
        if n < START_SCENE:
            continue
        if END_SCENE is not None and n > END_SCENE:
            continue
        target.append(p)

    if not target:
        print("처리할 비디오가 없습니다. (START_SCENE/END_SCENE, 파일명 Scene-XXX 형식 확인)")
        return

    end_label = END_SCENE if END_SCENE is not None else "end"
    range_result_path = os.path.join(
        OUTPUT_PATH, f"all_captions_scene_{START_SCENE:03d}_{end_label}.json"
    )
    results = {}

    pbar = tqdm(target, desc=f"Captions | Scene {START_SCENE}~{end_label}", unit="video")
    for video_path in pbar:
        scene_number = extract_scene_number(str(video_path))
        caption = generate_video_caption(str(video_path), scene_number)

        # JSON 파싱 & 저장
        try:
            caption_json = parse_caption_to_json(caption)
            results[video_path.name] = caption_json

            out_file = os.path.join(OUTPUT_PATH, f"{video_path.stem}_caption.json")
            with open(out_file, "w", encoding="utf-8") as f:
                json.dump(caption_json, f, ensure_ascii=False, indent=2)

            pbar.set_postfix_str("")

        except Exception:
            # 파싱 실패면 txt 저장
            out_file = os.path.join(OUTPUT_PATH, f"{video_path.stem}_caption.txt")
            with open(out_file, "w", encoding="utf-8") as f:
                f.write(caption)

            pbar.set_postfix_str("json_parse_fail")

        time.sleep(SLEEP_BETWEEN)

    # 범위 전체 결과 저장
    with open(range_result_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print("\n완료")
    print(f"Range 결과: {range_result_path}")
    print(f"개별 결과 폴더: {OUTPUT_PATH}")

if __name__ == "__main__":
    process_videos_range()